# Building Up Process Scheduling Program
#### Queue1 = Round Robin List take values that has High Priority = 0 with quantum time = 3
#### Queue2 = SRJ take values that has medium Priority = 1 with quantum time = 2
#### Qeuee3 = SJF take values that has low Priority = 2 without quantum time as it's non-premptive 
# Needed Inputs
#### 1.Process Name (Pi)
#### 2.Process Arival Time 
#### 3.Prcoess Execution Time
#### 4.Queue Number of Priority


In [70]:
def SRT(processData):
    n = len(processData)

    arrival_time = [process[1] for process in processData]
    burst_time = [process[2] for process in processData]
    waiting_time = [0] * n

    quantum_time = 2

    remaining_time = [0] * n
    for i in range(n):
        remaining_time[i] = burst_time[i]

    t = 0  # Current time
    completed = 0  # Number of completed processes
    while completed != n:
        shortest = -1
        for i in range(n):
            if arrival_time[i] <= t and remaining_time[i] > 0:
                if shortest == -1 or remaining_time[i] < remaining_time[shortest]:
                    shortest = i

        if shortest == -1:
            t += 1
            continue

        if remaining_time[shortest] > quantum_time:
            t += quantum_time
            remaining_time[shortest] -= quantum_time
        else:
            t += remaining_time[shortest]
            waiting_time[shortest] = t - burst_time[shortest] - arrival_time[shortest]
            remaining_time[shortest] = 0
            completed += 1

    total_waiting_time = sum(waiting_time)
    average_waiting_time = total_waiting_time / n

    print("\nProcess\t\tArrival Time\tBurst Time\tWaiting Time")
    for i in range(n):
        print(
            "{}\t\t{}\t\t{}\t\t{}".format(
                processData[i][0], arrival_time[i], burst_time[i], waiting_time[i]
            )
        )

    print("\nAverage Waiting Time:", average_waiting_time)

In [74]:
def SJF(process_data):
    total_turnaround_time = 0
    total_waiting_time = 0
    start_time = []
    exit_time = []
    s_time = 0
    process_data.sort(key=lambda x: x[1])
    """
        Sort processes according to the Arrival Time
    """
    for i in range(len(process_data)):
        ready_queue = []
        temp = []
        normal_queue = []

        for j in range(len(process_data)):
            if (process_data[j][1] <= s_time) and (process_data[j][3] == 2):
                temp.extend(
                    [process_data[j][0], process_data[j][1], process_data[j][2]]
                )
                ready_queue.append(temp)
                temp = []
            elif process_data[j][3] == 2:
                temp.extend(
                    [process_data[j][0], process_data[j][1], process_data[j][2]]
                )
                normal_queue.append(temp)
                temp = []

        if len(ready_queue) != 0:
            ready_queue.sort(key=lambda x: x[2])
            """
                Sort the processes according to the Burst Time
            """
            start_time.append(s_time)
            s_time = s_time + ready_queue[0][2]
            e_time = s_time
            exit_time.append(e_time)
            for k in range(len(process_data)):
                if process_data[k][0] == ready_queue[0][0]:
                    break
            process_data[k][3] = 1
            process_data[k].append(e_time)

        elif len(ready_queue) == 0:
            if s_time < normal_queue[0][1]:
                s_time = normal_queue[0][1]
            start_time.append(s_time)
            s_time = s_time + normal_queue[0][2]
            e_time = s_time
            exit_time.append(e_time)
            for k in range(len(process_data)):
                if process_data[k][0] == normal_queue[0][0]:
                    break
            process_data[k][3] = 1
            process_data[k].append(e_time)

    for i in range(len(process_data)):
        turnaround_time = process_data[i][4] - process_data[i][1]
        """
            turnaround_time = completion_time - arrival_time
        """
        total_turnaround_time = total_turnaround_time + turnaround_time
        process_data[i].append(turnaround_time)
    average_turnaround_time = total_turnaround_time / len(process_data)
    """
        average_turnaround_time = total_turnaround_time / no_of_processes
    """

    for i in range(len(process_data)):
        waiting_time = process_data[i][5] - process_data[i][2]
        """
            waiting_time = turnaround_time - burst_time
        """
        total_waiting_time = total_waiting_time + waiting_time
        process_data[i].append(waiting_time)
    average_waiting_time = total_waiting_time / len(process_data)
    """
        average_waiting_time = total_waiting_time / no_of_processes
    """
    print("Process ID\tArrival Time\tBurst Time\tTurnaround Time\tWaiting Time")
    for process in process_data:
        print(
            f"{process[0]}\t\t{process[1]}\t\t{process[2]}\t\t{process[5]}\t\t{process[6]}"
        )

    print(f"\nAverage Turnaround Time: {round(average_turnaround_time)}")
    print(f"Average Waiting Time: {round(average_waiting_time)}")

In [72]:
def processData(no_of_processes):
    processSRT = []
    processSJF = []
    for i in range(no_of_processes):
        temporary = []
        process_id = int(input("Enter Process ID: "))

        arrival_time = int(input(f"Enter Arrival Time for Process {process_id}: "))

        burst_time = int(input(f"Enter Burst Time for Process {process_id}: "))
        
        priority_rank=int(input(f"Enter priority rank for Process {process_id}: "))
        
        temporary.extend([process_id, arrival_time, burst_time, priority_rank])
        if priority_rank==1:
            processSJF.append(temporary)
        if priority_rank==2:
            processSRT.append(temporary)


    return processSRT,processSJF

In [75]:
srt,sjf=processData(4)

ValueError: invalid literal for int() with base 10: ''

In [ ]:
SRT(srt)

In [ ]:
SJF(sjf)

IndexError: list index out of range